# LAB 3.1 - CNS (TDNN and RNN)

Import of libraries, fix of random seed and device.


In [1]:
import json
import os
from pathlib import Path
import random
import numpy as np
import matplotlib.pyplot as plt
import itertools
from typing import Callable
from tqdm.notebook import tqdm
import torch
from torchvision import datasets

seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

device = 'cuda'

Loading of dataset NARMA10.

In [2]:
with open(f'NARMA10.csv') as file:
    data = file.read().split('\n')[:-1]
    X_narma10 = torch.Tensor([float(r) for r in data[0].split(',')])
    Y_narma10 = torch.Tensor([float(r) for r in data[1].split(',')])

X_narma10.shape, Y_narma10.shape

(torch.Size([10000]), torch.Size([10000]))

Split in train and test set for each X and Y data.

In [3]:
tr_size = 5000

TR_X_narma10 = X_narma10[:tr_size]
TR_Y_narma10 = Y_narma10[:tr_size]
TS_X_narma10 = X_narma10[tr_size:]
TS_Y_narma10 = Y_narma10[tr_size:]

TR_X_narma10.shape, TR_Y_narma10.shape, TS_X_narma10.shape, TS_Y_narma10.shape

(torch.Size([5000]),
 torch.Size([5000]),
 torch.Size([5000]),
 torch.Size([5000]))

Functions able to provide an easiest way to plot the results.

In [4]:
def _base_plot(
        elements: list[tuple],
        title: str = '',
        xlabel: str = '',
        ylabel: str = '',
        xscale=None,
        yscale=None,
        save_name=None
) -> None:
    """Function able to plot a list of elements."""
    plt.figure(figsize=(20, 10))
    plt.title(title)
    for line, label in elements:
        plt.plot(line, label=label)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    if xscale is not None:
        plt.xscale(xscale)
    if yscale is not None:
        plt.yscale(yscale)
    plt.grid()
    plt.legend()
    if save_name is not None:
        plt.savefig(f'tdnn_and_rnn_plots/{save_name}.png')
    plt.show()


def plot_tr(
        data: torch.Tensor,
        prediction: torch.Tensor,
        save_name: str = None
) -> None:
    """Function able to plot the training y data and predictions to make a comparison between them."""
    if save_name is not None:
        save_name = f'{save_name}_train'
    prediction = prediction.reshape(-1).tolist()
    _base_plot(
        [
            (data[-len(prediction):], 'time series'),
            (prediction, 'prediction'),
        ],
        title='Time series of training set',
        xlabel='X',
        ylabel='Y',
        save_name=save_name,
        xscale='log',
    )


def plot_ts(
        data: torch.Tensor,
        prediction: torch.Tensor,
        save_name: str = None
) -> None:
    """Function able to plot the test y data and predictions to make a comparison between them."""
    if save_name is not None:
        save_name = f'{save_name}_test'
    prediction = prediction.reshape(-1).tolist()
    _base_plot(
        [
            (data[-len(prediction):], 'time series'),
            (prediction, 'prediction'),
        ],
        title='Time series of test set',
        xlabel='X',
        ylabel='Y',
        save_name=save_name,
        xscale='log',
    )


def plot_loss(
        tr_loss: list,
        ts_loss: list,
        save_name: str = None
) -> None:
    """Function able to plot the learning curve of training and test losses."""
    if save_name is not None:
        save_name = f'{save_name}_loss'
    _base_plot(
        [
            (tr_loss, 'TR loss'),
            (ts_loss, 'TS loss'),
        ],
        title='Loss (MSE) for epochs',
        xlabel='Epochs',
        ylabel='Loss (MSE)',
        save_name=save_name,
        yscale='log',
    )


def general_plot(
        train_loss: list,
        ts_loss: list,
        TR_Y: torch.Tensor,
        train_pred: torch.Tensor,
        TS_Y: torch.Tensor,
        eval_pred: torch.Tensor,
        save_name: str = None
) -> None:
    """Function able to plot all results of subplots related to learning curve, training data and test data."""
    plot_loss(train_loss, ts_loss, save_name=save_name)
    plot_tr(data=TR_Y, prediction=train_pred[-1], save_name=save_name)
    plot_ts(data=TS_Y, prediction=eval_pred[-1], save_name=save_name)

Train function able to fit a model given in input.

In [5]:
def train(
        model: torch.nn.Module,
        TR: tuple[torch.Tensor, torch.Tensor],
        TS: tuple[torch.Tensor, torch.Tensor],
        epochs: int = 10,
        sgd_config: dict = {},
        tqdm=None,
        save_name: str = None,
        device: str = 'cpu',
) -> tuple:
    """
    Function able to train a given model.

    model: Model to train.
    TR: Tuple composed by X train and Y train torch tensors.
    TS: Tuple composed by X test and Y test torch tensors.
    epochs: Number of epochs of training.
    sgd_config: Dictionary containing sgd configurations (lr and momentum).
    tqdm: TQDM object to show the progressbar. It is None when progressbar is not shown.
    save_name: Name given to save the torch model and data in a file. If it is None the file is not created.
    device: Name of device to use for computation.

    returns:
        tuple: Results of training. In particular the tuple is composed by 4 variables:
            - train_loss: List of MSE loss of training set computed for each epoch.
            - test_loss: List of MSE loss of test set computed for each epoch.
            - train_preds: List of predictions of training set for each epoch.
            - test_preds: List of predictions of test set for each epoch.
    """
    model.to(device)
    optimizer = torch.optim.SGD(model.parameters(), **sgd_config).to(device)
    criterion = torch.nn.MSELoss()
    train_loss, test_loss = [], []
    train_preds, test_preds = [], []
    X_TR, Y_TR = TR
    X_TS, Y_TS = TS
    model.eval()
    test_loss.append(criterion(model(X_TS), Y_TS).item())

    iterable = range(epochs)
    if tqdm is not None:
        iterable = tqdm(iterable)
    for _ in iterable:
        model.train()
        optimizer.zero_grad()
        pred_tr = model(X_TR.to(device))
        loss_tr = criterion(pred_tr, Y_TR.to(device))
        loss_tr.backward()
        optimizer.step()
        train_loss.append(loss_tr.item())
        train_preds.append(pred_tr)

        model.eval()
        pred_vl = model(X_TS.to(device))
        test_loss.append(criterion(pred_vl, Y_TS.to(device)).item())
        test_preds.append(pred_vl)

    train_loss.append(criterion(model(X_TR), Y_TR).item())
    if save_name is not None:
        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': train_loss,
            'test_loss': test_loss[-1],
        }, f'tdnn_and_rnn_variables/{save_name}_model_and_tr_info.pt')
    return train_loss, test_loss, train_preds, test_preds

Gridsearch function able to find the best configuration for a model created in a `train_func` function callback, train the model with the best configuration and test it on test set.

In [6]:
def gridsearch(
        train_func: Callable,
        configs: dict,
        TR: tuple[torch.Tensor, torch.Tensor],
        TS: tuple[torch.Tensor, torch.Tensor],
        epochs: int = 100,
        vl_tr_size: int = 4000,
        attempts_for_config: int = 1,
        save_name: str = None,
        device: str = 'cpu',
) -> tuple:
    """
    Gridsearch function able to find the best hyperparameters configuration, train the model with the best config and test it.

    train_func: Function able to create a model and train it given a config, a train and validation set and a number of epochs.
    configs: Hyperparameters configurations to investigate to find the best one that minimizes the loss on validation set. In particular this is a dictionary of lists for each hyperparam to investigate that is transformed by this function in a list of dictionaries.
    TR: Training set data (X, Y).
    TS: test set data (X, Y).
    epochs: Number of epochs of training both for model selection and model evaluation.
    vl_tr_size: Number of example to use in training set of model selection phase. It is useful to split training set in training and validation set.
    attempts_for_config: Number of attempts to do for each configuration. The loss that it's minimized is the mean of each loss of each attempt.
    save_name: Name used for saving the file related to mse info (mse of training, validation and test set).
    device: Name of device to use for computation.

    returns: A tuple of 4 variables related to the result of training function during the model evaluation phase (training on entire training set and test on test set).
    """
    configs = [dict(zip(configs.keys(), t)) for t in itertools.product(*configs.values())]
    best_config = {}
    best_loss = None
    X_TR, Y_TR = TR
    for i, config in enumerate(tqdm(configs)):
        vl_loss = 0
        for j in range(attempts_for_config):
            _, tdnn_eval_loss, _, _ = train_func(
                config,
                (X_TR[:vl_tr_size], Y_TR[:vl_tr_size]),
                (X_TR[vl_tr_size:], Y_TR[vl_tr_size:]),
                epochs=epochs,
                device=device,
            )
            vl_loss += tdnn_eval_loss[-1]
        vl_loss /= attempts_for_config
        print(f'{i + 1}/{len(configs)} - Tried config {config} with loss {vl_loss}')
        if best_loss is None or vl_loss < best_loss:
            best_config = config
            best_loss = vl_loss
    print(f'Best config: {best_config} with loss {best_loss}')
    print('Retraining...')
    train_loss, eval_loss, train_preds, eval_preds = train_func(
        best_config,
        TR,
        TS,
        epochs=epochs,
        tqdm=tqdm,
        save_name=save_name,
        device=device,
    )

    if save_name is not None:
        mse_data = {
            'train_mse': train_loss[-1],
            'validation_mse': best_loss,
            'test_mse': eval_loss[-1]
        }
        print(mse_data)
        with open(f'tdnn_and_rnn_variables/{save_name}_mse_data.json', 'w') as file:
            json.dump(mse_data, file)

    return train_loss, eval_loss, train_preds, eval_preds

### TDNN

TDNN pytorch model.

In [7]:
class TDNN(torch.nn.Module):
    """
    Class of TDNN model.
    """

    def __init__(self, window: int, hidden_size: int, output_size: int, n_layers: int = 1) -> None:
        """
        TDNN constructor method.

        window: Window of TDNN that is related to the number of previous time steps seen in the past.
        hidden_size: Size of hidden state.
        output_size: Size of output value.
        n_layers: Number of hidden layers. Default this is 1.
        """
        super(TDNN, self).__init__()
        hidden_layers = [torch.nn.Linear(window + 1, hidden_size)]
        for _ in range(n_layers - 1):
            hidden_layers.append(torch.nn.Linear(hidden_size, hidden_size))
        self.hidden_layers = torch.nn.Sequential(*hidden_layers)
        self.output_layer = torch.nn.Linear(hidden_size, output_size)

    def forward(self, X: torch.Tensor) -> torch.Tensor:
        """
        Forward function used to the forward phase of pytorch module.

        X: Input data.

        returns:
            torch.Tensor: Output data.
        """
        for hidden_layer in self.hidden_layers:
            preactivation = hidden_layer(X)
            X = torch.nn.functional.tanh(preactivation)
        return self.output_layer(X)


TDNN(10, 100, 1, n_layers=2)

TDNN(
  (hidden_layers): Sequential(
    (0): Linear(in_features=11, out_features=100, bias=True)
    (1): Linear(in_features=100, out_features=100, bias=True)
  )
  (output_layer): Linear(in_features=100, out_features=1, bias=True)
)

TDNN train function.

In [8]:
def train_tdnn(
        config: dict,
        TR: torch.Tensor,
        TS: torch.Tensor,
        epochs: int = 10,
        tqdm=None,
        save_name: str = None,
        device: str = 'cpu',
) -> tuple:
    """
    Function used to train the TDNN model. It wraps the general train function.

    config: Dictionary of hyperparameters.
    TR: Training set.
    TS: Test set.
    epochs: Number of epochs.
    tqdm: Object used to show the progressbar.
    save_name: Able to provide the title of file containing pytorch model and training info.
    device: Name of device to use for computation.

    returns:
        tuple: Train results.
    """
    window = config['window']
    model = TDNN(window, config['hidden_size'], 1, n_layers=config['n_layers'])
    sets = []
    for X, Y in [TR, TS]:
        X = torch.unsqueeze(X.unfold(0, window + 1, 1), 0)
        Y = Y[window:].reshape(1, -1, 1)
        sets.append((X, Y))
    return train(model, sets[0], sets[1], epochs=epochs, tqdm=tqdm, save_name=save_name, device=device, sgd_config={
        'lr': config['lr'],
        'momentum': config['momentum'],
    })

TDNN gridsearch and results.

In [9]:
tdnn_save_name = 'tdnn_narma10'

tdnn_train_loss, tdnn_ts_loss, tdnn_train_pred, tdnn_eval_pred = gridsearch(
    train_func=train_tdnn,
    configs=dict(
        window=[8, 10, 12],
        hidden_size=[100],
        n_layers=[1, 2],
        lr=[0.1, 0.01],
        momentum=[0.5, 0.2],
    ),
    TR=(TR_X_narma10, TR_Y_narma10),
    TS=(TS_X_narma10, TS_Y_narma10),
    epochs=500,
    vl_tr_size=4000,
    attempts_for_config=5,
    save_name=tdnn_save_name,
    device=device,
)

general_plot(tdnn_train_loss, tdnn_ts_loss, TR_Y_narma10, tdnn_train_pred, TS_Y_narma10, tdnn_eval_pred,
             save_name=tdnn_save_name)

  0%|          | 0/24 [00:00<?, ?it/s]

AssertionError: Torch not compiled with CUDA enabled

### RNN

RNN pytorch model.

In [ ]:
class RNN(torch.nn.Module):
    """
    Class of RNN model.
    """

    def __init__(
            self,
            input_size: int,
            hidden_size: int,
            output_size: int,
            stateful: bool = True,
            n_layers: int = 1
    ) -> None:
        """
        RNN constructor method.

        input_size: Size of input value.
        hidden_size: Size of hidden state.
        output_size: Size of output value.
        stateful: Boolean set to true if it's want to use the final training hidden state as initial hidden state of evaluation.
        n_layers: Number of hidden layers. Default this is 1.
        """
        super(RNN, self).__init__()
        self.recoursive_layer = torch.nn.RNN(input_size, hidden_size, num_layers=n_layers)
        self.output_layer = torch.nn.Linear(hidden_size, output_size)
        self.stateful = stateful
        self.h = None

    def forward(self, X: torch.Tensor) -> torch.Tensor:
        """
        Forward function used to the forward phase of pytorch module.

        X: Input data.

        returns:
            torch.Tensor: Output data.
        """
        state, h = self.recoursive_layer(X, self.h)
        if self.stateful and self.training:
            self.h = h.detach()
        return self.output_layer(state)


RNN(1, 100, 1)

RNN train function.

In [ ]:
def train_rnn(
        config: dict,
        TR: tuple[torch.Tensor, torch.Tensor],
        TS: tuple[torch.Tensor, torch.Tensor],
        epochs: int = 10,
        tqdm=None,
        save_name: str = None,
        device: str = 'cpu',
) -> tuple:
    """
    Function used to train the RNN model. It wraps the general train function.

    config: Dictionary of hyperparameters.
    TR: Training set.
    TS: Test set.
    epochs: Number of epochs.
    tqdm: Object used to show the progressbar.
    save_name: Able to provide the title of file containing pytorch model and training info.
    device: Name of device to use for computation.

    returns:
        tuple: Train results.
    """
    model = RNN(TR[0].shape[-1], config['hidden_size'], TR[1].shape[-1], n_layers=config['n_layers'])
    return train(model, TR, TS, epochs=epochs, tqdm=tqdm, save_name=save_name, device=device, sgd_config={
        'lr': config['lr'],
        'momentum': config['momentum'],
    })

RNN gridsearch and results.

In [ ]:
rnn_save_name = 'rnn_narma10'

rnn_train_loss, rnn_ts_loss, rnn_train_pred, rnn_eval_pred = gridsearch(
    train_func=train_rnn,
    configs=dict(
        hidden_size=[100],
        n_layers=[1, 2],
        lr=[0.1, 0.01],
        momentum=[0.5, 0.2],
    ),
    TR=(TR_X_narma10.reshape(-1, 1, 1), TR_Y_narma10.reshape(-1, 1, 1)),
    TS=(TS_X_narma10.reshape(-1, 1, 1), TS_Y_narma10.reshape(-1, 1, 1)),
    epochs=500,
    vl_tr_size=4000,
    attempts_for_config=3,
    save_name=rnn_save_name,
    device=device,
)

general_plot(rnn_train_loss, rnn_ts_loss, TR_Y_narma10, rnn_train_pred, TS_Y_narma10, rnn_eval_pred,
             save_name=rnn_save_name)

# Bonus track 1 - Mackey-glass 17 task

Mackey-glass 17 task dataset loading and split in train and test set.

In [ ]:
tr_size = 5000

with open(f'MG17.csv') as file:
    data = file.read().split('\n')[:-1][0]
    data = torch.Tensor([float(r) for r in data.split(',')])
    TR_mg17 = data[:tr_size]
    TS_mg17 = data[tr_size:]

TR_X_mg17 = TR_mg17[:-1]
TR_Y_mg17 = TR_mg17[1:]
TS_X_mg17 = TS_mg17[:-1]
TS_Y_mg17 = TS_mg17[1:]

TR_X_mg17.shape, TR_Y_mg17.shape, TS_X_mg17.shape, TS_Y_mg17.shape

### TDNN

TDNN gridsearch and results.

In [ ]:
tdnn_save_name = 'tdnn_mg17'

tdnn_train_loss, tdnn_ts_loss, tdnn_train_pred, tdnn_eval_pred = gridsearch(
    train_func=train_tdnn,
    configs=dict(
        window=[8, 10, 12],
        hidden_size=[100],
        n_layers=[1, 2],
        lr=[0.1, 0.01],
        momentum=[0.5, 0.2],
    ),
    TR=(TR_X_mg17, TR_Y_mg17),
    TS=(TS_X_mg17, TS_Y_mg17),
    epochs=100,
    vl_tr_size=4000,
    attempts_for_config=5,
    save_name=tdnn_save_name,
    device=device,
)

general_plot(tdnn_train_loss, tdnn_ts_loss, TR_Y_mg17, tdnn_train_pred, TS_Y_mg17, tdnn_eval_pred,
             save_name=tdnn_save_name)

### RNN

RNN gridsearch and results.

In [ ]:
rnn_save_name = 'rnn_mg17'

rnn_train_loss, rnn_ts_loss, rnn_train_pred, rnn_eval_pred = gridsearch(
    train_func=train_rnn,
    configs=dict(
        hidden_size=[100],
        n_layers=[1, 2],
        lr=[0.1, 0.01],
        momentum=[0.5, 0.2],
    ),
    TR=(TR_X_mg17.reshape(-1, 1, 1), TR_Y_mg17.reshape(-1, 1, 1)),
    TS=(TS_X_mg17.reshape(-1, 1, 1), TS_Y_mg17.reshape(-1, 1, 1)),
    epochs=100,
    vl_tr_size=4000,
    attempts_for_config=3,
    save_name=rnn_save_name,
    device=device,
)

general_plot(rnn_train_loss, rnn_ts_loss, TR_Y_mg17, rnn_train_pred, TS_Y_mg17, rnn_eval_pred,
             save_name=rnn_save_name)

# Bonus track 2 - Sequential MNIST classification task

Function able to download and get tensors related to MNIST data and labels of train and test set.

In [ ]:
def download_mnist() -> tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
    """
    Function able to download MNIST dataset and return it.

    returns:
        tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]: Training data and labels and test data and labels of MNIST dataset.
    """
    mnist_dir = 'MNIST/'
    if not Path(mnist_dir).exists():
        os.mkdir(mnist_dir)
    TR_MNIST = datasets.MNIST(root=f'{mnist_dir}', train=True, download=True, transform=None)
    TS_MNIST = datasets.MNIST(root=f'{mnist_dir}', train=False, download=True, transform=None)
    TR_DATA_MNIST = TR_MNIST.train_data.reshape(-1, 28 * 28).type(torch.float32)
    TS_DATA_MNIST = TS_MNIST.test_data.reshape(-1, 28 * 28).type(torch.float32)
    TR_LABELS_MNIST = torch.nn.functional.one_hot(TR_MNIST.train_labels).type(torch.float32)
    TS_LABELS_MNIST = torch.nn.functional.one_hot(TS_MNIST.test_labels).type(torch.float32)
    return TR_DATA_MNIST, TR_LABELS_MNIST, TS_DATA_MNIST, TS_LABELS_MNIST


TR_DATA_MNIST, TR_LABELS_MNIST, TS_DATA_MNIST, TS_LABELS_MNIST = download_mnist()

TR_DATA_MNIST.shape, TR_LABELS_MNIST.shape, TS_DATA_MNIST.shape, TS_LABELS_MNIST.shape

Functions able to compute accuracy and plot a list of training and test accuracy for each epoch.

In [ ]:
def accuracy(out: torch.Tensor, pred: torch.Tensor) -> float:
    """
    Function that compute accuracy given an output and prediction tensor.

    out: Output tensor.
    pred: Prediction tensor.

    returns:
        float: Computed accuracy value.
    """
    return (sum(pred.argmax(-1) - out.argmax(-1) == 0) / len(out)).item()


def plot_accuracy(train_accuracy: list, test_accuracy: list, save_name: str = None) -> None:
    """
    Function able to plot the accuracy for each epoch.

    train_accuracy: List of train accuracy for each epoch.
    test_accuracy: List of test accuracy for each epoch.
    """
    if save_name is not None:
        save_name = f'{save_name}_accuracy'
    _base_plot(
        [
            (train_accuracy, 'TR accuracy'),
            (test_accuracy, 'TS accuracy'),
        ],
        title='Accuracy for epochs',
        xlabel='Epochs',
        ylabel='Accuracy',
        save_name=save_name,
    )

Function able to perform RNN gridsearch and plot of results related to MSE loss and accuracy.

In [ ]:
def perform_rnn_gs_and_plot(TR: tuple[torch.Tensor, torch.Tensor], TS: tuple[torch.Tensor, torch.Tensor], save_name: str) -> None:
    """
    Function able to perform RNN gridsearch and plot of results related to MSE loss and accuracy.

    TR: Training set.
    TS: Test set.
    save_name: Name given to the file where is saved the model.
    """
    rnn_train_loss, rnn_ts_loss, rnn_train_pred, rnn_eval_pred = gridsearch(
        train_func=train_rnn,
        configs=dict(
            hidden_size=[100],
            n_layers=[1, 2],
            lr=[0.1, 0.01],
            momentum=[0.9, 0.2],
        ),
        TR=TR,
        TS=TS,
        epochs=50,
        vl_tr_size=4000,
        attempts_for_config=1,
        save_name=save_name,
        device=device,
    )

    plot_loss(rnn_train_loss, rnn_ts_loss, save_name=save_name)

    tr_accuracy = [accuracy(TR[1], pred) for pred in rnn_train_pred]
    ts_accuracy = [accuracy(TS[1], pred) for pred in rnn_eval_pred]
    print(f'Accuracy: [TR: {tr_accuracy[-1]}] - [TS: {ts_accuracy[-1]}]')
    plot_accuracy(tr_accuracy,ts_accuracy,save_name=save_name)


Sequential MNIST model selection and model evaluation results.

In [ ]:
perform_rnn_gs_and_plot(
    (TR_DATA_MNIST, TR_LABELS_MNIST),
    (TS_DATA_MNIST, TS_LABELS_MNIST),
    save_name='rnn_seq_mnist'
)

MNIST dataset permutation od data to perform permuted sequential MNSIT.

In [ ]:
permutations = np.random.permutation(28 * 28)

TR_DATA_PMNIST = TR_DATA_MNIST[:, permutations]
TS_DATA_PMNIST = TS_DATA_MNIST[:, permutations]

TR_DATA_PMNIST.shape, TS_DATA_PMNIST.shape

Permuted MNIST model selection and model evaluation results.

In [ ]:
perform_rnn_gs_and_plot(
    (TR_DATA_PMNIST, TR_LABELS_MNIST),
    (TS_DATA_PMNIST, TS_LABELS_MNIST),
    save_name='rnn_permuted_mnist'
)